In [7]:
import pandas as pd
import json

# Path to the JSONL file
file_path = '../data/raw_data/results_galica_api_sample.jsonl'

# Read the JSONL file and convert it to a list of dictionaries
data = []
with open(file_path, 'r') as file:
    for line in file:
        json_line = json.loads(line)
        data.append(json_line)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

In [8]:
# process API results

df_filtered = df[['file_id','title', 'date', 'type', 'setSpec']].copy()
df_filtered = df_filtered.explode('setSpec')
df_filtered = df_filtered[df_filtered['setSpec'].str.contains("theme")]
df_filtered['setSpec'] = df_filtered['setSpec'].apply(lambda x : x.split('gallica:theme:')[1])
df_filtered['category_number'] = df_filtered['setSpec'].apply(lambda x : x.split(':')[0])
df_filtered['sub_category_number'] = df_filtered['setSpec'].apply(lambda x : x.split(':')[1])

df_filtered['category_number'] = df_filtered['category_number'].astype(int)
df_filtered['sub_category_number'] = df_filtered['sub_category_number'].astype(int)

In [9]:
# Merge with data mapping
df_mapping = pd.read_csv('../data/raw_data/galica_mapping_processed.csv')
df_final = pd.merge(df_filtered, df_mapping, on = ['category_number', 'sub_category_number'])
df_final['full_category_name'] = df_final['category_name'] + df_final['sub_category_name']
df_final = df_final.drop(['title', 'date'], axis=1)

data_raw_sample = pd.read_csv('../data/raw_data/galica_mon_sampling_title.csv', index_col = [0])
df_final = pd.merge(data_raw_sample, df_final, on = 'file_id', how = 'outer')
df_final = df_final[~df_final['date'].str.contains('-')]
df_final.to_csv('../data/data_preprocessed/merged_sample_data.csv')

